In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")
SERPER_API_KEY=os.getenv("SERPER_API_KEY")

In [3]:
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY
os.environ["GROQ_API_KEY"]=GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"]=GOOGLE_API_KEY
os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [8]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="llama-3.3-70b-versatile")

In [9]:
message=[{'role':'system','content':'you are a helpful assistant'},
         {'role':'user','content':'Hii how are you'}]

In [10]:
llm.invoke(message).content

"Hello. I'm doing well, thanks for asking. How can I assist you today? Is there something I can help you with or would you like to chat?"

In [23]:
class Chatbot():
    def __init__(self,system=""):
        self.system=system
        self.message=[]
        if self.system:
            self.message.append({'role':'system','content':system})

    def __call__(self,message):
        self.message.append({'role':'user','content':message})
        result=self.execute()
        self.message.append({'role':'assistant','content':result})
        return result

    def execute(self):
        llm=ChatGroq(model_name="Gemma2-9b-It")
        result=llm.invoke(self.message)
        return result.content

In [24]:
bot=Chatbot(system="You are a helpful assistant")

In [26]:
bot("Hi my name is Gyanbardhan")

"Hi Gyanbardhan, it's nice to meet you! 👋\n\nHow can I help you today? 😊  \n\n"

In [27]:
bot.message

[{'role': 'system', 'content': 'You are a helpful assistant'},
 {'role': 'user', 'content': 'Hi my name is Gyanbardhan'},
 {'role': 'assistant',
  'content': "Hi Gyanbardhan, it's nice to meet you! 👋\n\nHow can I help you today? 😊  \n\n"}]

In [29]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.


Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
you have to answer "hi","hello","i am good".
""".strip()

### Regular expression Pattern Breakdown:

- ^: This matches the start of a string. It means the string must begin with what follows.
- Action:: This is a literal match. It means the string must have the text "Action:" at the beginning.
- (\w+):
    - The parentheses () define a capture group. This allows you to extract part of the string that matches this section.
    - \w+ matches one or more word characters (letters, digits, and underscores). This will capture a word that follows "Action:".
- :: This is a literal colon that separates the word matched by (\w+) from the rest of the string.
- (.*): This is another capture group, where .* matches any character (.) zero or more times (*), which means it captures everything that comes after the second colon.
#### What does it do?
This regex is looking for a string that:

- Starts with the text "Action:".
- Has a word right after it, separated by a colon.
- Then, after another colon, it captures everything that follows.

In [14]:
import re
action_re=re.compile('^Action: (\w+): (.*)')

In [28]:
import httpx
def wikipedia(query):
    response=httpx.get("https://en.wikipedia.org/w/api.php",params={
        "action":"query",
        "list":"search",
        "srsearch":query,
        "format":"json"
    })
    return response.json()["query"]['search'][0]["snippet"]

In [30]:
import httpx
def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    return response.json()[0]["text"]

In [20]:
def calculate(number):
    return eval(number)

In [32]:
known_actions={
    "wikipedia":wikipedia,
    "calculate":calculate,
    "simon_blog_search":simon_blog_search,
}

In [22]:
def query(question,max_turns=5):
    i=0
    bot=Chatbot(prompt)
    next_prompt=question
    while i<max_turns:
        pass

In [33]:
def query(question,max_turns=5):
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result

In [34]:
query("Capital of France")

Thought: I need to find the capital of France.

Action: wikipedia: France
PAUSE 

 -- running wikipedia France
Observation: <span class="searchmatch">France</span>, officially the <span class="searchmatch">French</span> Republic, is a country located primarily in Western Europe. Its overseas regions and territories include <span class="searchmatch">French</span> Guiana in
Thought:  I need to find the capital city, it's mentioned somewhere in this text.

Action: wikipedia: France 
PAUSE 

 -- running wikipedia France 
Observation: <span class="searchmatch">France</span>, officially the <span class="searchmatch">French</span> Republic, is a country located primarily in Western Europe. Its overseas regions and territories include <span class="searchmatch">French</span> Guiana in
Thought:  Hmm, I'm not seeing the capital city in this snippet. I need to look for more information.

Action: wikipedia: France 
PAUSE 

 -- running wikipedia France 
Observation: <span class="searchmatch">France<

'Answer: The capital of France is Paris. \n'

In [39]:
query("Is simons written about AI")

Thought: I need to search Simon's blog to see if he has written about AI.  
Action: simon_blog_search: AI 
PAUSE  

 -- running simon_blog_search AI 
Observation: It's OK to call it Artificial Intelligence: Update 9th January 2024: This post was clumsily written and failed to make the point I wanted it to make. I've published a follow-up, What I should have said about the term Artificial Intelligence which you should read instead.
My original post follows.


We need to be having high quality conversations about AI: what it can and can't do, its many risks and pitfalls and how to integrate it into society in the most beneficial ways possible.
Any time I write anything that mentions AI it's inevitable that someone will object to the very usage of the term.
Strawman: "Don't call it AI! It's not actually intelligent - it's just spicy autocomplete."
That strawman is right: it's not "intelligent" in the same way that humans are. And "spicy autocomplete" is actually a pretty good analogy for 

'Thought: Simon has written about AI and provided a link to a follow-up post. \n\nAnswer: Yes, Simon has written about AI and suggests reading a follow-up post for his more developed thoughts on the topic.  \n'